In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

from torchnlp.datasets import penn_treebank_dataset
import torch
from torchnlp.samplers import BPTTBatchSampler
from torch.utils.data import DataLoader
from rsm_samplers import MNISTSequenceSampler, ptb_pred_sequence_collate
from ptb import lang_util
from util import activity_square
from torchvision import datasets, transforms
import torch.nn.functional as F
import torchvision.utils as vutils
from torch.nn import CrossEntropyLoss, MSELoss
from importlib import reload 
from torch.utils.data import Sampler, BatchSampler
import rsm
from functools import reduce, partial
import matplotlib.pyplot as plt

In [2]:
reload(lang_util)
corpus = lang_util.Corpus('/Users/jgordon/nta/datasets/PTB')

print(corpus.read_out(corpus.train[:60]))

aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter <eos> pierre <unk> N years old will join the board as a nonexecutive director nov. N <eos> mr. <unk> is chairman of <unk> n.v. the dutch publishing group <eos> rudolph <unk> N years old and former


In [6]:
def _plot_timeseries(inputs, memory, layers=2):
    steps = len(memory)
    fig, axs = plt.subplots(layers + 1, steps, dpi=144, figsize=(9, 3.3))
    for step in range(steps):
        for l in range(layers):
            ax = axs[l][step]
            ax.imshow(activity_square(memory[step][l, 0]))
            ax.axis('off')
        input_ax = axs[layers][step]
        input_ax.imshow(activity_square(inputs[step]))
        input_ax.axis('off')
    plt.show()
    

In [22]:
import rsm_samplers
import rsm
from ptb import lang_util
import rsm_experiment
reload(rsm_samplers)
reload(rsm)
reload(lang_util)
reload(rsm_experiment)

CONFIG = {
    'path': "~/nta/results",
    'data_dir': "~/nta/datasets",
    'predictor_hidden_size': 200,
    'predictor_output_size': 10000,
    'batch_size': 300,
    'batches_in_epoch': 500,
    'vocab_size': 10000,
    'embed_dim': 28,
    'eval_interval': 3,
    'eval_batches_in_epoch': 270,
    
    'm': 500,
    'n': 1,
    'k=': 10,
    'eps': [0.0, 0.0],
    'gamma': 0.0,
    'forget_mu': 0.02,
    'k_winner_cells': 1,
    'weight_sparsity': None,
    
    # Hierarchy
    'n_layers': 2,
    'mult_integration': True,
    'feedback': True,
    'tp_boosting': False,
    'int_loss_decay': 0.5,
    'loss_layers': 'all',
}

exp = rsm_experiment.RSMExperiment(config=CONFIG)
exp.model_setup(CONFIG)

setup: Using cpu
Created <RSMLayer m=200 n=6 k=25 d_in=784 eps=0.00 /> with 3194384 trainable params
Created <RSMLayer m=200 n=6 k=25 d_in=1200 eps=0.00 /> with 3361200 trainable params
Created RSMNet with 2 layer(s)


In [23]:
input_timeseries = []
memory_timeseries = []
TIMESTEPS = 10

def fwd_hook(mod, input_tuple, output_tuple):
    inputs, prior_hidden = input_tuple
    out, hidden = output_tuple
    x_b = hidden[0]
    input_timeseries = ([inputs[0]] + input_timeseries)[:TIMESTEPS]
    memory_timeseries = ([x_b] + memory_timeseries)[:TIMESTEPS]
    _plot_timeseries(input_timeseries[::-1], memory_timeseries[::-1], layers=n_layers)

# exp.model.register_forward_hook(fwd_hook)

EPOCHS = 10

for epoch in range(EPOCHS):
    exp.train_epoch(epoch)

KeyboardInterrupt: 

#### Confirming cross-epoch sequence continuity

In [ ]:
collate_fn = partial(rsm_samplers.ptb_pred_sequence_collate, vector_dict=bwe.embedding_dict)

sampler = rsm_samplers.PTBSequenceSampler(corpus.train, batch_size=10, max_batches=1)
loader = DataLoader(corpus.train,
                       batch_sampler=sampler,
                       collate_fn=collate_fn)


EPOCHS = 50

last_output = None

sequence = []
for i in range(EPOCHS):
    total_loss = 0.0
    input_timeseries = []
    memory_timeseries = []

    for batch_id, (inputs, target, pred_target, pred_input) in enumerate(loader):
        sequence.append(pred_input[0].item())

print(corpus.read_out(sequence))
print(len(sequence))